In [3]:
import pandas as pd
import numpy as np

In [1]:
# Import Data
from data import OverallGeneralEcon

     time  Econ_GrowthRate  ...  National_Income  National_Income_growth_rate
0   103Q1             4.69  ...          3458937                         6.67
1   103Q2             4.92  ...          3469291                         9.05
2   103Q3             4.90  ...          3607436                         7.46
3   103Q4             4.38  ...          3676529                         7.71
4   104Q1             4.80  ...          3793426                         9.67
5   104Q2             1.89  ...          3589202                         3.46
6   104Q3            -0.28  ...          3654406                         1.30
7   104Q4            -0.20  ...          3823611                         4.00
8   105Q1            -0.09  ...          3826530                         0.87
9   105Q2             1.69  ...          3709202                         3.34
10  105Q3             3.00  ...          3761737                         2.94
11  105Q4             3.92  ...          4007984                